In [7]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Paths
DATA_PATH = "D:\Projects\Edge-Optimized ECG Arrhythmia Detection using TensorFlow Lite and Quantization\data\ecg.csv"
MODEL_DIR = "models/ecg_saved_model"

def load_data():
    print("Loading ECG dataset...")
    df = pd.read_csv(DATA_PATH)

    X = df.iloc[:, :-1].values.astype(np.float32)
    y = df.iloc[:, -1].values.astype(np.int32)

    return train_test_split(X, y, test_size=0.2, random_state=42)

def build_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

def main():
    X_train, X_test, y_train, y_test = load_data()

    print("Building model...")
    model = build_model(input_shape=X_train.shape[1], num_classes=5)

    print("Training model...")
    model.fit(X_train, y_train, epochs=5, batch_size=32)

    print("Evaluating model...")
    loss, acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {acc:.4f}")

    print("Saving model in SavedModel format...")
    os.makedirs("models", exist_ok=True)
    model.export(MODEL_DIR)

    print("SavedModel exported successfully!")

if __name__ == "__main__":
    main()


Loading ECG dataset...
Building model...
Training model...
Epoch 1/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8834 - loss: 0.4378  
Epoch 2/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 944us/step - accuracy: 0.9251 - loss: 0.2671
Epoch 3/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 929us/step - accuracy: 0.9386 - loss: 0.2189
Epoch 4/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 910us/step - accuracy: 0.9454 - loss: 0.1928
Epoch 5/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 964us/step - accuracy: 0.9507 - loss: 0.1772
Evaluating model...
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9475 - loss: 0.1915  
Test Accuracy: 0.9475
Saving model in SavedModel format...
INFO:tensorflow:Assets written to: models/ecg_saved_model\assets


INFO:tensorflow:Assets written to: models/ecg_saved_model\assets


Saved artifact at 'models/ecg_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 187), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2295903783440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2296397602960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2296397601808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2296397597392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2296397603536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2296397601232: TensorSpec(shape=(), dtype=tf.resource, name=None)
SavedModel exported successfully!


In [8]:
import os
import tensorflow as tf

MODEL_DIR = "models/ecg_saved_model"
TFLITE_PATH = "models/ecg_model.tflite"
TFLITE_QUANT_PATH = "models/ecg_model_quant.tflite"

def convert_model():
    print("Converting SavedModel → TFLite...")

    converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

    # Normal conversion
    tflite_model = converter.convert()
    with open(TFLITE_PATH, "wb") as f:
        f.write(tflite_model)

    print("Normal TFLite model saved:", TFLITE_PATH)

    # Quantized conversion
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    quant_model = converter.convert()

    with open(TFLITE_QUANT_PATH, "wb") as f:
        f.write(quant_model)

    print("Quantized TFLite model saved:", TFLITE_QUANT_PATH)

def main():
    os.makedirs("models", exist_ok=True)
    convert_model()

if __name__ == "__main__":
    main()


Converting SavedModel → TFLite...
Normal TFLite model saved: models/ecg_model.tflite
Quantized TFLite model saved: models/ecg_model_quant.tflite


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

DATA_PATH = "D:\Projects\Edge-Optimized ECG Arrhythmia Detection using TensorFlow Lite and Quantization\data\ecg.csv"
MODEL_PATH = "models/ecg_model_quant.tflite"

def main():
    print("Loading quantized TFLite model...")
    interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("Loading sample ECG input...")
    df = pd.read_csv(DATA_PATH)

    X = df.iloc[:, :-1].values.astype(np.float32)
    y = df.iloc[:, -1].values.astype(np.int32)

    sample = X[0]
    sample_label = y[0]

    sample = np.expand_dims(sample, axis=0)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], sample)

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output)

    print("Actual Label:", sample_label)
    print("Predicted Label:", predicted_class)

if __name__ == "__main__":
    main()


Loading quantized TFLite model...
Loading sample ECG input...


d:\Projects\Edge-Optimized ECG Arrhythmia Detection using TensorFlow Lite and Quantization\venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


FileNotFoundError: [Errno 2] No such file or directory: 'data/ecg.csv'